In [1]:
'''Loading libraries'''
import pandas as pd
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
import os 
import math

#path = "C:/Users/raoki/Documents/Project 1/"
path = 'C:/Users/raque/Google Drive/SFU/CMPT884 - Course Project and presentation'
os.chdir(path)
%matplotlib inline  

In [2]:
'''Loading Logistic Regresson Features by gene'''
#Pre-prossing on R
gsd = pd.read_table('data_gscore_features_bygene.csv', sep=';',index_col=False)
gsd.head()

,gene,ensembl_gene_id,gene_biotype,chromosome_name,start,end,g_score,mutsig,avg_expr,sirna
0,A1BG,ENSG00000121410,protein_coding,19,58345178,58353499,0.027891,-0.257187,5.419211,0.449420
1,A1CF,ENSG00000148584,protein_coding,10,50799409,50885675,0.009572,-0.012908,0.559459,-0.693753
2,A2M,ENSG00000175899,protein_coding,12,9067664,9116229,0.009709,-0.250799,13.373874,0.372179
3,A2ML1,ENSG00000166535,protein_coding,12,8822472,8887001,0.011142,-0.001836,3.841615,1.745096
4,A3GALT2,ENSG00000184389,protein_coding,1,33306766,33321098,0.022153,NaN,NaN,NaN


In [3]:
'''Normalization Features'''
#The bigger the value, higher the chance of being a driver gene
#print(features.head())
gsd['mutsig'] = gsd['mutsig']*(-1) 
gsd['sirna'] = gsd['sirna']*(-1)

gsd['mutsig'] = (gsd['mutsig']-gsd['mutsig'].mean())/(gsd['mutsig'].std())
gsd['avg_expr'] = (gsd['avg_expr']-gsd['avg_expr'].mean())/(gsd['avg_expr'].std())
gsd['sirna'] = (gsd['sirna']-gsd['sirna'].mean())/(gsd['sirna'].std())

In [4]:
'''Droping SiRNA and NA'''
print(gsd.shape)
gsd = gsd.dropna(axis=0,how='any')
print(gsd.shape)

(19279, 10)
(12957, 10)


In [5]:
'''Selecting top100 genes in each feature'''
#The top100 genes will be added into the final dataset, 
#even if their g-score is not very large

#mutsig
gsd = gsd.sort_values('mutsig', ascending=0)
top_genes_f = gsd['gene'].head(100)
#avg_expr
gsd = gsd.sort_values('avg_expr', ascending=0)
top_genes_f = top_genes_f.append(gsd['gene'].head(100))
#sirna
gsd = gsd.sort_values('sirna', ascending=0)
top_genes_f = top_genes_f.append(gsd['gene'].head(100))

#removing duplicated genes on the vector 
top_genes_f = top_genes_f.unique()
top_genes_f = set(top_genes_f)

'''Evaluation'''
intogen = ['ADCY1','AHNAK','AKAP9','APC','AQR','ARFGAP3','ARID1B','ATIC','ATM','ATRX','BCLAF1',
           'BCOR','BNC2','BPTF','BRAF','CASP1','CAT','CDC27','CDH1','CDKN1B','CEP290','CHD1L',
           'CHD3','CHD4','CHEK2','CNOT1','CNOT3','CNTNAP1','CTNNB1','CUL2','CUL3','EEF1B2','EGFR',
           'EIF2AK3','EIF4G1','EP300','ERCC2','FAT1','FGFR2','FIP1L1','FN1','FRG1','G3BP2','GNAS',
           'HGF','HNF1A','HRAS','HSP90AB1','HSPA8','IDH1','IRS2','KDM6A','KEAP1','MECOM','MED12',
           'MLL2','MYH10','NAP1L1','NKX3-1','NOTCH1','NOTCH2','NUP98','PCDH18','PIK3CB','PLXNA1',
           'PRPF8','PTEN','RPSAP58','SCAI','SETDB1','SMAD4','SMARCA1','SMARCB1','SPOP','SVEP1','TAOK2',
           'TBL1XR1','TBX3','THRAP3','TJP1','TJP2','TP53','TP53BP1','TRIO','WHSC1L1','WNT5A','ZFHX3','ZNF814']


census = ['ACSL3','AR','AXIN1','BRAF','CANT1','DDX5','ELK4','ERG','ETV1','ETV4','ETV5',
          'FOXA1','HERPUD1','HNRNPA2B1','KLF6','KLK2','NCOR2','NDRG1','PTEN','RAF1',
          'SALL4','SLC45A3','SPOP','TMPRSS2','ZFHX3']

evaluation = list(set(intogen) | set(census))

#forcing those genes to appear on the dataset
#top_genes_f = set.union(top_genes_f, evaluation)

'''This genes have high values from features, they should be at the data after
the filter, even if their g-score are not large'''
#Bool variable: 1 if gene is on top100 from features, 0 otherwise 
gsd['top_features'] = 0
gsd = gsd.reset_index(drop=True)

for i in np.arange(0,gsd.shape[0]):  
    if gsd.gene[i] in top_genes_f:
        gsd.loc[i,'top_features'] = 1

In [6]:
'''Defining driver genes model's seed'''
#top100 genes from g-score are used as driver genes seed
#top101-1500 genes from g-score or present in the top100 features are the passengers genes seed 
#The threhold 1500 was defined based on the paper (they mentioned 84 regions with about 14 genes each)
gsd = gsd.sort_values(by=['g_score'],axis=0,ascending=False)
gsd['top_gscore'] = 0
gsd['top_gscore'][101:1200] = 1

'''Creating data'''
#y: 1 if gene is driver, 0 otherwise 
gd = gsd[0:100]
gd['y']=1
gp = gsd.loc[((gsd.top_gscore==1) | (gsd.top_features==1))]
gp['y']=0

#new dataset deleting unimportant columns 
#some warnings, but everything is fine  
f = 4
data = pd.concat([gd,gp])
data['intercept']=1
data = data[['gene','g_score','intercept','mutsig','avg_expr','sirna','y']]
data.columns = ['gene','gscore','intercept','mutsig','avg_expr','sirna','y']

C:\Users\raque\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\raque\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\raque\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 

In [7]:
'''Checking g-score distribution of driver and passengers genes'''
print('driver genes size - seed',gd['g_score'].shape[0])
print('passenger genes size - seed',gp['g_score'].shape[0])

driver genes size - seed 100
passenger genes size - seed 1369


In [8]:
'''Metropolis-Hastings algorithm'''
#Definitions 
#theta: set of unknown paramets
#using normal distribution instead of exponential distribution

#Unknown parameters - start values
#N(mu0,var0): driver genes
#N(mu1,var1): passenger genes
theta_start = [0,0,0,0,0.06,0.06] #w0,w1,w2,w3,mu0, mu1
T = np.array(data['y'])

'''Likelihood 1: P(T|w,x)~logit()'''
def likelihood1_old(param,data,T,f):
    xw = data[['intercept','mutsig','avg_expr','sirna']].dot(param[0:f])
    l1 = 1/(1+np.exp(-xw))
    l0 = 1 - l1  
    l1[T==0] = l0[T==0]
    l1 = l1+0.000001
    return np.log(l1).sum()

'''Likelihood 2: P(Gscore|T,mean,sd)~Normal(mean,sd)'''
def likelihood2_old(param,T,data,f):
    media = T
    media=[param[f] if x==0 else param[f+1] for x in media]
    l2 = scipy.stats.norm(media,0.1).pdf(data.gscore)
    return np.log(l2).sum()

#print(likelihood1(theta_start, data, T),likelihood2(theta_start, T, data,f))

In [9]:
'''Priori: w0,w1,w2,w3, mu0,mu1'''
def priori(param,f):
    w = scipy.stats.norm(0,0.25).pdf(param[0:f])
    mu0 = scipy.stats.norm(0.05,0.03).pdf(param[f])
    mu1 = scipy.stats.norm(0.09,0.03).pdf(param[f+1])
    r = np.log(w).sum()+np.log(mu1)+np.log(mu0)
    return r


In [10]:
'''Posterior'''
def posterior(param,T,data,f):
    #print(likelihood2_old(param,T,data,f),likelihood1_old(param,data,T,f),priori(param,f),np.around(param,2))
    return likelihood2_old(param,T,data,f)+likelihood1_old(param,data,T,f)+priori(param,f)

#Vector with proposed values 
# Values are proposed from a Normal distribution
# mean is equal to current values and variance are hyperparameters 
#theta_proposed = np.random.normal(theta_current,[0.5,0.5,0.5,0.5,0.02,0.02,0.0004,0.0004])
'''ProposalValue'''
def proposalvalues(param):
    return(np.random.normal(param,[0.05,0.05,0.05,0.05,0.005,0.005]))

In [11]:
def T_update_old(param,f):
    xw = data[['intercept','mutsig','avg_expr','sirna']].dot(param[0:f])
    t11 = 1/(1+np.exp(-xw)) #array    
    t21 = scipy.stats.norm(param[f+1], 0.01).pdf(data.gscore)

    t10 = 1-(1/(1+np.exp(-xw))) #array    
    t20 = scipy.stats.norm(param[f], 0.01).pdf(data.gscore)
    
    t1 = t11*t21
    t0 = t10*t20
    
    prob_t1 = t1/(t1+t0)
    
    t_new = np.random.binomial(1,prob_t1,len(prob_t1))
    #print(t_new.sum(),'\n')
    return t_new


In [12]:
'''MCMC'''
def runMCMC_old(startvalue,iterations,T,data,f):
    chain = np.zeros((iterations+1,len(startvalue)))
    Tmatrix = np.zeros((iterations+1,len(T)))
    chain[0]=startvalue
    Tmatrix[0]=T
    for i in np.arange(iterations):
        proposal = proposalvalues(chain[i,])
        prob = np.exp(posterior(proposal,T,data,f)-posterior(chain[i,],T,data,f))
        #print('\n',i)
        if np.all(np.random.uniform(0,1,1) < prob):
            chain[i+1] = proposal
            T = T_update_old(proposal,f) 
            Tmatrix[i+1] = T
        else:
            chain[i+1] = chain[i]
            Tmatrix[i+1] = Tmatrix[i]
    return chain, T ,Tmatrix       
            

#https://theoreticalecology.wordpress.com/2010/09/17/metropolis-hastings-mcmc-in-r/

In [ ]:
f=4 #3 features + intercept
chain, Tnew,Tmatrix  = runMCMC_old(theta_start,8000,T,data,f)


In [ ]:
np.savetxt('simplified_chain2_T.csv',chain,delimiter=';')
np.savetxt('simplified_matrix_T.csv',Tmatrix,delimiter=';')


In [ ]:
print('acceptance rate',chain.shape, np.unique(chain,axis=0).shape,np.unique(chain,axis=0).shape[0]*100/chain.shape[0])

chain2 = pd.DataFrame(chain, columns=['w0','w1','w2','w3','mu0','mu1'])
chain2['iteration'] = chain2.index
chain2.head()
plt.plot(chain2.iteration,chain2.w0,'r--',
         chain2.iteration,chain2.w1,'b--',
         chain2.iteration,chain2.w2,'y--',
         chain2.iteration,chain2.w3,'g--')
plt.show()
plt.plot(chain2.iteration,chain2.mu0,'r--',
         chain2.iteration,chain2.mu1,'b--')
plt.show()


In [ ]:
print('last iteration T=1 elements',Tnew.sum())

Tmatrix = Tmatrix[int(Tmatrix.shape[0]*0.25):Tmatrix.shape[0]]
tprob = Tmatrix.sum(axis=0)/Tmatrix.shape[0]
#tfinal = np.random.binomial(1,tprob,len(tprob))
tfinal = tprob
#tfinal[tprob==1] = 1
#tfinal[tprob<1] = 0
tfinal = np.random.binomial(1,tprob,len(tprob))

gd = data[tfinal==0]
gp = data[tfinal==1]
print('driver genes size - final',gd['gscore'].shape[0])
print('passenger genes size - final',gp['gscore'].shape[0])

In [ ]:
maximum = list(set(data.gene.tolist()).intersection(evaluation))

gene = gd.gene.tolist()
driver = list(set(gene).intersection(maximum))
print('Proportion of Driver from other lists (Recall): ', len(driver)*100/len(maximum),'%(',len(driver),'/',len(maximum),')')
print('Proportion of Driver Genes on Data: ',len(gene)*100/data.shape[0],'%(',len(gene),')')
print('Proportion of Driver Genes on other lists (Precision):',len(driver)*100/len(gene),'%')
#driver

In [ ]:
pd.options.display.max_rows = 4000
gene = gd.gene.tolist()
#gene

In [ ]:
top100 = data
top100['prob'] = tprob
top100 = top100.sort_values("prob", axis=0, ascending=False)
genes100 = top100['gene'].head(100)
#list(genes100)

In [ ]:
driver = list(set(genes100).intersection(maximum))
print('Proportion of Driver from other lists (Recall): ', len(driver)*100/len(maximum),'%(',len(driver),'/',len(maximum),')')
print('Proportion of Driver Genes on Data: ',len(genes100)*100/data.shape[0],'%(',len(genes100),')')
print('Proportion of Driver Genes on other lists (Precision):',len(driver)*100/len(genes100),'%')
#driver